In [ ]:
!pip install "pyiceberg[s3fs,hive]"

In [ ]:
! pip install kafka-python

In [125]:
from kafka import KafkaConsumer
import json
try:
        # Create KafkaConsumer
        consumer = KafkaConsumer(
            "apmlogs",
            bootstrap_servers=['telemetry-kafka-external-bootstrap-observability-kafka.apps.zagaopenshift.zagaopensource.com:443'],
            value_deserializer=lambda v: v,  # Keep value as bytes for JSON decoding
            group_id="apmingestion",
            auto_offset_reset='earliest',
            consumer_timeout_ms=10000,
            security_protocol='SSL',  # Add SSL configuration if needed
            ssl_cafile='./telemetry_zagaopenshift.pem',
            ssl_certfile='./telemetry_zagaopenshift.crt'
        )
        print("Kafka consumer created successfully.")
except Exception as e:
        print(f"Error creating Kafka consumer: {e}")

Kafka consumer created successfully.


In [126]:
    messages=[]
    try:
        for message in consumer:
            json_message = json.loads(message.value.decode('utf-8'))
            messages.append(json_message)
            if(len(messages) > 5 ):
              consumer.close()
    except Exception as e:
        print(f"Error consuming messages: {e}")
    finally:
        consumer.close()
        print("Message consumption completed.")

ERROR:kafka.consumer.fetcher:Fetch to node 0 failed: Cancelled: <BrokerConnection node_id=0 host=telemetry-kafka-external-0-observability-kafka.apps.zagaopenshift.zagaopensource.com:443 <connected> [IPv4 ('59.91.155.113', 443)]>


Message consumption completed.


In [127]:
for msg in messages:
  print(msg)

{'resourceLogs': [{'resource': {'attributes': [{'key': 'container.id', 'value': {'stringValue': 'a0bf9fa1ba205cac19ae9584e73de46d3c8de0cbe0be787f6a216eead1c0003b'}}, {'key': 'host.arch', 'value': {'stringValue': 'amd64'}}, {'key': 'host.name', 'value': {'stringValue': 'kafka-srv-5-59bfc54888-2gkzx'}}, {'key': 'k8s.container.name', 'value': {'stringValue': 'kafka-srv-5'}}, {'key': 'k8s.deployment.name', 'value': {'stringValue': 'kafka-srv-5'}}, {'key': 'k8s.namespace.name', 'value': {'stringValue': 'observability-testcases'}}, {'key': 'k8s.node.name', 'value': {'stringValue': 'zagaocp-worker2'}}, {'key': 'k8s.pod.name', 'value': {'stringValue': 'kafka-srv-5-59bfc54888-2gkzx'}}, {'key': 'k8s.replicaset.name', 'value': {'stringValue': 'kafka-srv-5-59bfc54888'}}, {'key': 'os.description', 'value': {'stringValue': 'Linux 5.14.0-284.71.1.el9_2.x86_64'}}, {'key': 'os.type', 'value': {'stringValue': 'linux'}}, {'key': 'process.command_args', 'value': {'arrayValue': {'values': [{'stringValue': 

In [48]:
from pyiceberg.catalog import load_catalog
catalog = load_catalog(
        "docs",
    **{
            "uri": "thrift://hive-ms-trino.apps.zagaopenshift.zagaopensource.com:9888",
            "s3.endpoint": "http://minio-lb.apps.zagaopenshift.zagaopensource.com:9009",
            "py-io-impl": "pyiceberg.io.pyarrow.PyArrowFileIO",
            "s3.access-key-id": "minioAdmin",
            "s3.secret-access-key": "minio1234",
    }
)

In [224]:
namespace="docs_logs" #enter namespace
ns=catalog.list_namespaces()
print(ns)
if((namespace,) in ns):
  print("Namespace exists")
else:
  catalog.create_namespace(namespace)
  print("Namespace create suceesss")



[('data',), ('default',), ('demo_data',), ('demo_saro',), ('demo_trace',), ('docs_check',), ('docs_logs',), ('docs_mari',), ('docs_powerleveldata',), ('docs_sa',), ('docs_sar',), ('docs_saro',), ('docs_selva',), ('hi',), ('hlo',), ('ingesting',), ('new',), ('old',), ('poc',), ('pod',), ('pod_data',), ('power_pod',), ('powerlevelpoddata',), ('powerpod',), ('saravan',), ('saravana',), ('saro',), ('saro_data',), ('sh',), ('sharanya',), ('sk',), ('soh',), ('trace_ingestion_data',)]
Namespace exists


In [74]:
# catalog.drop_namespace("docs_logs")

In [218]:
from pyiceberg.schema import Schema
from pyiceberg.types import (
    TimestampType,
    LongType,
    IntegerType,
    FloatType,
    DoubleType,
    StringType,
    NestedField,
    StructType,
    ListType
)
from pyiceberg.partitioning import PartitionSpec, PartitionField
from pyiceberg.transforms import DayTransform
from pyiceberg.table.sorting import SortOrder, SortField
from pyiceberg.transforms import IdentityTransform

# Define the schema for the spans
schema = Schema(
    NestedField(field_id=1, name="servicename", field_type=StringType(), required=False),
    NestedField(field_id=2, name="language", field_type=StringType(), required=False),
    NestedField(field_id=3, name="logsdata", field_type=StringType(), required=False),
    NestedField(field_id=4, name="createdTime", field_type=TimestampType(), required=False),

)

# partition_spec = PartitionSpec(
#     PartitionField(
#         source_id=1, field_id=1000, transform=DayTransform(), name="datetime_day"
#     )
# )

sort_order = SortOrder(SortField(source_id=3, transform=IdentityTransform()))

table_name = "logs"  # Define your table identifier


table = catalog.create_table(
    identifier=(namespace, table_name),
    schema=schema,
    location="s3a://apmlogs/logs", #mention s3 bucketname
    # partition_spec=partition_spec,
    sort_order=sort_order,
)
print(f"Table '{table_name}' created successfully.")



Table 'logs' created successfully.


In [216]:
catalog.list_tables("docs_logs")  #mention namespace

[]

In [215]:
catalog.drop_table("docs_logs.logs") #mention namespace and tablename

In [219]:
from pyiceberg.io.pyarrow import PyArrowFileIO
import pyarrow as pa
import json
import datetime

format=[]

for data in messages:
  for key in data['resourceLogs'][0]['resource']['attributes'] :

    if(key['key']=='service.name'):
      final={
          "servicename":key['value']['stringValue'],
      }
    if(key['key']=='telemetry.sdk.language'):
      final.update({
          "language":key['value']['stringValue'],
          "logsdata":json.dumps(data),
          "createdTime":datetime.datetime.now()
      })
      format.append(final)

# print(format)
df=pa.Table.from_pylist(format)
table.append(df)
print("Data appended successfully.")






Data appended successfully.


In [220]:
table.scan().to_pandas()

,servicename,language,logsdata,createdTime
0,kafka-srv-5,java,"{""resourceLogs"": [{""resource"": {""attributes"": ...",2024-07-23 09:50:34.385348
1,kafka-srv-5,java,"{""resourceLogs"": [{""resource"": {""attributes"": ...",2024-07-23 09:50:34.385471
2,kafka-srv-5,java,"{""resourceLogs"": [{""resource"": {""attributes"": ...",2024-07-23 09:50:34.385583
3,kafka-srv-5,java,"{""resourceLogs"": [{""resource"": {""attributes"": ...",2024-07-23 09:50:34.385707
4,kafka-srv-5,java,"{""resourceLogs"": [{""resource"": {""attributes"": ...",2024-07-23 09:50:34.385824
5,kafka-srv-5,java,"{""resourceLogs"": [{""resource"": {""attributes"": ...",2024-07-23 09:50:34.385933


In [222]:
logs = catalog.load_table("docs_logs.logs") #Query the data

In [223]:
logs.scan().to_pandas() #Query the data pandas

,servicename,language,logsdata,createdTime
0,kafka-srv-5,java,"{""resourceLogs"": [{""resource"": {""attributes"": ...",2024-07-23 09:50:34.385348
1,kafka-srv-5,java,"{""resourceLogs"": [{""resource"": {""attributes"": ...",2024-07-23 09:50:34.385471
2,kafka-srv-5,java,"{""resourceLogs"": [{""resource"": {""attributes"": ...",2024-07-23 09:50:34.385583
3,kafka-srv-5,java,"{""resourceLogs"": [{""resource"": {""attributes"": ...",2024-07-23 09:50:34.385707
4,kafka-srv-5,java,"{""resourceLogs"": [{""resource"": {""attributes"": ...",2024-07-23 09:50:34.385824
5,kafka-srv-5,java,"{""resourceLogs"": [{""resource"": {""attributes"": ...",2024-07-23 09:50:34.385933
